<a href="https://colab.research.google.com/github/YooshinCho/SKhynix_defect_detection/blob/master/Defect_Detection_tutorial_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 첫 번째 신경망 훈련하기: 기초적인 분류 문제

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
</table>

이번 튜토리얼은 우리가 딥러닝을 통해서 불량 검출을 하였을 때, 모델이 실제로 잘 불량을 검출하고 있는지 확인하는 방법을 배우게 될 것입니다. 불량 검출은 환경에 따라서 여러 가지 어려운 조건이 붙을 수 도 있지만, 간단하게는 2 class classification이라고 볼 수 있습니다. 하지만, 실제로 잘 불량이 검출되는지를 확인하기 위해서 test accuracy만을 이용하기 보다 CAM(class activation map)이라는 방법을 사용하려고 합니다. CAM은 Bolei Zhou의 Learning Deep Features for Discriminative Localization 논문에서 제시된 방법입니다.





CAM은 모델에 input image에 대해서 어떤 특정 class로 분류를 할 때 어떤 부분에 activation을 많이 하고 있는지를 visualization 해주는 기법입니다. 이를 위해서 상당히 간단하고 직관적이지만 효과적인 모델 구조를 가지고 있습니다. Feature map을 global average pooling한 다음 fully connectd layer를 통과시켜 최종 아웃풋을 뽑아내는 방법인대요, 이를 통해서 각 feature map이 얼마나 기여를 하고 있는지 fully connected layer의 weight를 통해서 알 수 있는 것입니다.

우선 오늘의 튜토리얼의 순서는 다음과 같습니다.

1. 패션 mnist 데어터셋 임포트

2. 데이터 전처리

3. 모델 구성

4. 모델 훈련 및 평가

5. CAM 영상 확인


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

import cv2

print(tf.__version__)

## 패션 MNIST 데이터셋 임포트하기

10개의 범주(category)와 70,000개의 흑백 이미지로 구성된 [패션 MNIST](https://github.com/zalandoresearch/fashion-mnist) 데이터셋을 사용하겠습니다. 이미지는 해상도(28x28 픽셀)가 낮고 다음처럼 개별 옷 품목을 나타냅니다:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>그림 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">패션-MNIST 샘플</a> (Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

패션 MNIST는 컴퓨터 비전 분야의 "Hello, World" 프로그램격인 고전 [MNIST](http://yann.lecun.com/exdb/mnist/) 데이터셋을 대신해서 자주 사용됩니다. MNIST 데이터셋은 손글씨 숫자(0, 1, 2 등)의 이미지로 이루어져 있습니다. 여기서 사용하려는 옷 이미지와 동일한 포맷입니다.

패션 MNIST는 일반적인 MNIST 보다 조금 더 어려운 문제이고 다양한 예제를 만들기 위해 선택했습니다. 두 데이터셋은 비교적 작기 때문에 알고리즘의 작동 여부를 확인하기 위해 사용되곤 합니다. 코드를 테스트하고 디버깅하는 용도로 좋습니다.

네트워크를 훈련하는데 60,000개의 이미지를 사용합니다. 그다음 네트워크가 얼마나 정확하게 이미지를 분류하는지 10,000개의 이미지로 평가하겠습니다. 패션 MNIST 데이터셋은 텐서플로에서 바로 임포트하여 적재할 수 있습니다:

In [0]:
BATCH_SIZE = 256
EPOCHS = 10
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

뒤에서 쓰인 hyper-parameter입니다. 

1. BATCH_SIZE는 256, EPOCHS는 10 으로 지정해주세요.

load_data() 함수를 호출하면 네 개의 넘파이(NumPy) 배열이 반환됩니다:

* `train_images`와 `train_labels` 배열은 모델 학습에 사용되는 *훈련 세트*입니다.
* `test_images`와 `test_labels` 배열은 모델 테스트에 사용되는 *테스트 세트*입니다.

이미지는 28x28 크기의 넘파이 배열이고 픽셀 값은 0과 255 사이입니다. *레이블*(label)은 0에서 9까지의 정수 배열입니다. 이 값은 이미지에 있는 옷의 *클래스*(class)를 나타냅니다:

<table>
  <tr>
    <th>레이블</th>
    <th>클래스</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

각 이미지는 하나의 레이블에 매핑되어 있습니다. 데이터셋에 *클래스 이름*이 들어있지 않기 때문에 나중에 이미지를 출력할 때 사용하기 위해 별도의 변수를 만들어 저장합니다:

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 데이터 전처리

네트워크를 훈련하기 전에 데이터를 전처리해야 합니다. 훈련 세트에 있는 첫 번째 이미지를 보면 픽셀 값의 범위를 확인하고 적당한 값으로 바꾸어줘어야 합니다.  우선 import 한 데이터셋의 픽셀값이 어떤 범위를 가지고 있는지 최댓값과 최솟값을 확인해주세요. 
([ ] 부분에 괄호를 지우고 알맞은 코드를 적어주세요)

1. 최댓값, 최솟값 확인 

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

print("max pixel value: ", train_images.max()) #1
print("min pixel value: ", train_images.min()) #2

신경망 모델에 주입하기 전에 이 값의 범위를 0~1 사이로 조정하겠습니다. 이렇게 하려면 255로 나누어야 합니다. *훈련 세트*와 *테스트 세트*를 동일한 방식으로 전처리하는 것이 중요합니다. 그리고 convolution filter를 이용할 것이기 때문에 뒤에 채널 수 1을 만들어주어야 합니다.

1. 픽셀 값의 범위를 0~1로 조정합니다.

2. 데이터의 shape를 (data_size, height, weight, channel)로 맞춰줍니다.


In [0]:
train_images = train_images / 255.0   
test_images = test_images / 255.0 


train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')

In [0]:
Defected_class = 1

train_labels_defect = train_labels /1
for i in range(0, train_labels_defect.shape[0]):
    if train_labels[i] == Defected_class:
        train_labels_defect[i] = 0
    else:
        train_labels_defect[i] = 1
        
test_labels_defect = test_labels /1
for i in range(0, test_labels_defect.shape[0]):
    if test_labels[i] == Defected_class:
        test_labels_defect[i] = 0
    else:
        test_labels_defect[i] = 1

*훈련 세트*에서 처음 25개 이미지와 그 아래 클래스 이름을 출력해 보죠. 데이터 포맷이 올바른지 확인하고 네트워크 구성과 훈련할 준비를 마칩니다.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i,:,:,0], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## 모델 구성

신경망 모델을 만들려면 모델의 층을 구성한 다음 모델을 컴파일합니다.

### 층 설정

신경망의 기본 구성 요소는 *층*(layer)입니다. 층은 주입된 데이터에서 표현을 추출합니다. 아마도 문제를 해결하는데 더 의미있는 표현이 추출될 것입니다.

대부분 딥러닝은 간단한 층을 연결하여 구성됩니다. 'keras.layers.Dense;,  'keras.layers.Conv2D'와 같은 층들의 가중치(parameter)는 훈련하는 동안 학습됩니다.

아래의 설명과 같은 구조의 모델을 정의해주세요.


1. input의 크기는 28*28*1 입니다.

2. Max pooling을 지날 때 height, weight가 절반이 됩니다. kernel의 크기는 2,2 입니다.

3. 모든 conv의 kernel size는 3,3 이고, stride는 1,1, 이고 activation은 relu 입니다.

4. layer의 순서를 다음 리스트를 통해 적을텐데 숫자는 conv layer를 의미하고 숫자는 해당 conv layer의 output channel의 숫자입니다. M은 max pooling을 의미합니다.

#[32, M, 64,  M, 128, 128]



In [0]:
def make_CAM_model():
    model = tf.keras.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same',
                                     input_shape=[28, 28, 1],activation = 'relu'))
    
    model.add(keras.layers.MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same',
                                     input_shape=[14, 14, 32],activation = 'relu'))
    
    model.add(keras.layers.MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(keras.layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same',
                                     input_shape=[7, 7, 64],activation = 'relu'))
    model.add(keras.layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same',
                                     input_shape=[7, 7, 128],activation = 'relu', name = 'final_conv'))
    
    model.add(keras.layers.GlobalAvgPool2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(10))
    model.add(keras.layers.Softmax())

    return model

In [0]:
model = make_CAM_model()


### 모델 컴파일

모델을 훈련하기 전에 필요한 몇 가지 설정이 모델 *컴파일* 단계에서 추가됩니다:

* *손실 함수*(Loss function)-훈련 하는 동안 모델의 오차를 측정합니다. 모델의 학습이 올바른 방향으로 향하도록 이 함수를 최소화해야 합니다.
* *옵티마이저*(Optimizer)-데이터와 손실 함수를 바탕으로 모델의 업데이트 방법을 결정합니다.
* *지표*(Metrics)-훈련 단계와 테스트 단계를 모니터링하기 위해 사용합니다. 다음 예에서는 올바르게 분류된 이미지의 비율인 *정확도*를 사용합니다.

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 모델 훈련

신경망 모델을 훈련하는 단계는 다음과 같습니다:

1. 훈련 데이터를 모델에 주입합니다-이 예에서는 `train_images`와 `train_labels` 배열입니다.
2. 모델이 이미지와 레이블을 매핑하는 방법을 배웁니다.
3. 테스트 세트에 대한 모델의 예측을 만듭니다-이 예에서는 `test_images` 배열입니다. 이 예측이 `test_labels` 배열의 레이블과 맞는지 확인합니다.

훈련을 시작하기 위해 `model.fit` 메서드를 호출하면 모델이 훈련 데이터를 학습합니다:

1. 넣어주어야 할  argument로는 train image, train label, 총 epoch 수. batch size가 있습니다.  괄호를 지우고 알맞은 코드를 넣어주세요.

In [0]:
model.fit(train_images, train_labels, epochs=EPOCHS, batch_size = BATCH_SIZE)

## 정확도 평가

그다음 테스트 세트에서 모델의 성능을 비교합니다:

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('테스트 정확도:', test_acc)

##CAM 영상 확인

이제 학습한 모델이 제대로 판단을 하고 있는지를 CAM을 그려보면서 확인할 것입니다. 필요한 것은 우선 가장 마지막 convolution filter의 output feature map이 필요하고, 저희가 원하는 class와 연결되는 fully connected layer의 weight가 필요합니다.
따라서 우선 'get_output_layer'라는 함수를 통해서 저희가 지정해준 'final_conv'를 찾아줍니다.

1. 반칸에 알맞는 dict를 써주세요.

In [0]:
def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

이제 'visualize_CAM' 함수를 채워보도록 하겠습니다. CAM 이미지를 만들기 위해서 가장 prediction이 높게 나온 class의 activation map을 만들어보도록 하겠습니다.

1. 마지막 fully connected layer의 weight가 필요함으로 마지막 fully connected layer에 접근하여 weight 값을 불러옵니다. (hint: model.layers[-1].get_weights()[0] 을 이용하면 가장 마지막 layers, 즉 softmax layer의 weight에 접근하게 됩니다.)

2. 위에서 정의한 'get_output_layer'와 저희가 정의해준 layer의 name을 통해서 마지막 convolution layer를 찾습니다. (hint : final_conv)

3. 우선 zero로 initialized 된 공백 이미지를 만듭니다. (hint : conv_output과 h,w가 같아야합니다.)

4. Fully connected layer의 weight로 가중치를 준 feature map을 공백에 더해줍니다.

5. feature map의 이미지가 원본 이미지와 크기가 다르므로 사이즈를 resize 시켜줍니다.



In [0]:
def visualize_CAM(model, img):
    _, width, height, _ = img.shape

    class_weights = model.layers[-2].get_weights()[0]
    
    final_conv_layer = get_output_layer(model, 'final_conv')
    get_output = keras.backend.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
    [conv_outputs, predictions] = get_output([img])
    conv_outputs = conv_outputs[0, :, :, :]
    predictions = np.argmax(predictions)

    
    #Create the class activation map.
    cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[0:2])
    for i, w in enumerate(class_weights[:, np.argmax(predictions)]):
            cam += w * conv_outputs[ :, :, i]
            
    print("predictions", class_names[predictions])
    cam = cam / np.max(cam)
    cam = cv2.resize(cam, (28,28))
    
    cam[np.where(cam < 0.2)] = 0
    
    img = img[0,:,:,0]
    img_cam = cam + img
    
    plt.figure(figsize=(10,10))
    plt.subplot(5,5,1)
    plt.imshow(cam, cmap=plt.cm.binary)
    plt.subplot(5,5,2)
    plt.imshow(img, cmap=plt.cm.binary)
    plt.subplot(5,5,3)
    plt.imshow(img_cam, cmap=plt.cm.binary)
    plt.show()
    

In [0]:
img = train_images[0]
img = np.expand_dims(img,0)
visualize_CAM(model, img)


In [0]:
def visualize_grad_CAM(model, img):
    
    _, width, height, _ = img.shape
    class_weights = model.layers[-2].get_weights()[0]
    final_conv_layer = get_output_layer(model, 'final_conv')
    
    
    with tf.GradientTape() as g:
        g.watch(final_conv_layer.output) 
        get_output = keras.backend.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
        
    print(final_conv_layer.output)
    
    [conv_outputs, predictions] = get_output([img])
    conv_grad = g.gradient(model.layers[-1].output, model.layers[0].input)    
    
    print(conv_grad)
    predictions = np.argmax(predictions)
    
    
    conv_outputs = conv_outputs[0, :, :, :]
    
    #Create the class activation map.
    cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[0:2])
    for i, w in enumerate(class_weights[:, np.argmax(predictions)]):
            cam += w * conv_outputs[ :, :, i]
    print("predictions", np.argmax(predictions))
    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = cv2.resize(cam, (28,28))

    
    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    
    #cam[np.where(cam < 0.2)] = 0
    
    print(cam.shape)
    print(img.shape)
    img = img[0,:,:,0]
    img = cam + img
    
    plt.imshow(img)
img = train_images[2]
img = np.expand_dims(img,0)

visualize_grad_CAM(model, img)